In [ ]:
import pandas as pd

In [ ]:
# data/253425.csvを読み込む
# 1行目はファンド名、2行目がヘッダー、3行目からデータ
# 文字コードはShift_JIS
df = pd.read_csv('data/253425.csv', encoding='shift_jis', header=1)

In [25]:
# 基準日、期間を指定するとその期間の週次リターンのdataframeを返す関数
# 基準日はYYYY-MM-DD形式、期間は週数
# 金曜日の基準価額から週次リターンを計算
# 引数は基準日、週数、基準価額のDataframe
def get_weekly_returns(base_date, weeks, df):
    # 基準日をdatetimeに変換
    base_date_dt = pd.to_datetime(base_date)
    # コピーして元データを保護
    df = df.copy()
    df['基準日'] = pd.to_datetime(df['基準日'])
    # 基準日以前のデータのみ使用（将来日付を除外）
    df_up_to = df.loc[df['基準日'] <= base_date_dt].sort_values('基準日')

    # 週ごとに、金曜日が無ければその週より過去で最も近い日を採用する
    selected_dates = []
    # 例: weeks=52 のとき、週の終わり日を過去から現在へ計算（古い順）
    for k in range(weeks, -1, -1):
        week_end = base_date_dt - pd.Timedelta(weeks=k)
        candidate = df_up_to.loc[df_up_to['基準日'] <= week_end, '基準日'].max()
        if pd.isnull(candidate):
            # この週より前にデータが存在しない場合はスキップ
            continue
        # 重複して同じ日付を複数回使わない
        if not selected_dates or candidate != selected_dates[-1]:
            selected_dates.append(candidate)

    if not selected_dates:
        # 利用可能な日付がない
        return pd.DataFrame(columns=['基準日', '週次リターン'])

    # 抽出した日付に対応する行を取得し、日付順にソート
    df_sel = df_up_to[df_up_to['基準日'].isin(selected_dates)].sort_values('基準日').copy()
    # 週次リターンを計算（前行との差分）
    df_sel['週次リターン'] = df_sel['基準価額（分配金再投資）(円)'].pct_change()
    # 最初の行はNaNになるため削除
    df_sel = df_sel.dropna(subset=['週次リターン'])
    # 要求された週数分（最新のもの）だけ返す
    if len(df_sel) > weeks:
        return df_sel[['基準日', '週次リターン']].tail(weeks)
    return df_sel[['基準日', '週次リターン']]

In [26]:
base_date = '2025-10-10'
weeks = 52
testdf = get_weekly_returns(base_date, weeks, df)

In [27]:
# get_weekly_returns関数を改造し、基準日と週数から週次リターンの標準偏差を返す関数
# 標準偏差を計算後、√52をかけて年率換算
def get_weekly_return_std(base_date, weeks, df):
    weekly_returns_df = get_weekly_returns(base_date, weeks, df)
    std_dev = weekly_returns_df['週次リターン'].std()
    annualized_std_dev = std_dev * (52 ** 0.5)
    return annualized_std_dev

In [30]:
# '2025-10-10'から過去26週、52週、156週の週次リターンの標準偏差を計算
# 表示は%表記で小数点以下3桁目を四捨五入
std_26 = get_weekly_return_std('2025-10-10', 26, df)
std_52 = get_weekly_return_std('2025-10-10', 52, df)
std_156 = get_weekly_return_std('2025-10-10', 156, df)  
print(f"過去26週の週次リターンの標準偏差: {std_26:.2%}")
print(f"過去52週の週次リターンの標準偏差: {std_52:.2%}")
print(f"過去156週の週次リターンの標準偏差: {std_156:.2%}")

過去26週の週次リターンの標準偏差: 10.03%
過去52週の週次リターンの標準偏差: 16.24%
過去156週の週次リターンの標準偏差: 14.91%
